<h1 style="color:grey">Multimedia Search and Retrieval: Group D</h1>

***

# 1. Fundamentals:
### Baseline System:
- Create a random retrieval system.
- Implement accuracy metrics (e.g., Precision@10, Recall@10, NDCG@10, MRR).
- Use genre as a relevance criterion.
### Text-Based Retrieval:
- Use TF-IDF representations of lyrics.
- Compute rankings using cosine similarity.
### User Interface:
- Build a simple web-based UI.
- Allow query selection and display retrieval results (optionally embed YouTube videos).

---
#### Preparatory cells

In [107]:
import pandas as pd
import random
import ast  # To parse string representations of lists
import numpy as np
from sklearn.metrics import ndcg_score

In [108]:
DATA_DIR = 'data/' 

#file paths
INFORMATION_FILE = DATA_DIR + 'id_information_mmsr.tsv'
GENRES_FILE = DATA_DIR + 'id_genres_mmsr.tsv'

#track information
information_df = pd.read_csv(INFORMATION_FILE, sep='\t')

#genres
genres_df = pd.read_csv(GENRES_FILE, sep='\t')

print(information_df.head())
print(genres_df.head())

                 id               artist              song  \
0  01rMxQv6vhyE1oQX  Against the Current    Chasing Ghosts   
1  02ZnlCGZEbkfCDxo        Laura Pausini  Tra Te E Il Mare   
2  04OjszRi9rC5BlHC         Grizzly Bear             Knife   
3  04iitW3ffa0mhpx3                Ne-Yo  Miss Independent   
4  04xUDjAYC14jsHyH           Jawbreaker     Jinx Removing   

                                    album_name  
0                                 In Our Bones  
1  The Best of Laura Pausini - E Ritorno Da Te  
2                                 Yellow House  
3  Year Of The Gentleman (Bonus Track Edition)  
4         24 Hour Revenge Therapy (Remastered)  
                 id                                              genre
0  01rMxQv6vhyE1oQX                               ['rock', 'pop punk']
1  02ZnlCGZEbkfCDxo  ['pop', 'italian pop', 'latin', 'europop', 'am...
2  04OjszRi9rC5BlHC  ['experimental', 'folk', 'lo fi', 'freak folk'...
3  04iitW3ffa0mhpx3  ['pop', 'r b', 'hip hop', 's

In [109]:
def parse_genres(genre_str):
    try:
        return ast.literal_eval(genre_str)
    except:
        return []

genres_df['genre'] = genres_df['genre'].apply(parse_genres)

catalog_df = pd.merge(information_df, genres_df, on='id', how='left')

# Handle missing genres if any
catalog_df['genre'] = catalog_df['genre'].apply(lambda x: x if isinstance(x, list) else [])

---
#### Subtask 1: Create a Simple Baseline System

In [110]:
def random_retrieval(query_track_id, catalog_df, N):
    """
    Randomly selects N tracks from the catalog, excluding the query track.

    Parameters:
    - query_track_id (str): The ID of the query track.
    - catalog_df (pd.DataFrame): The catalog containing all tracks.
    - N (int): Number of tracks to retrieve.

    Returns:
    - retrieved_tracks (pd.DataFrame): DataFrame of retrieved tracks.
    """
    #exclude the query track
    candidates = catalog_df[catalog_df['id'] != query_track_id]
    
    #randomly sample N tracks
    retrieved_tracks = candidates.sample(n=N, replace=False, random_state=random.randint(0, 1000000))
    
    return retrieved_tracks.reset_index(drop=True)

In [34]:
#example with the track i=0
query_track = catalog_df.iloc[0]
query_id = query_track['id']
N = 10

retrieved = random_retrieval(query_id, catalog_df, N)
print(retrieved[['id', 'artist', 'song', 'album_name']])

                 id                 artist                      song  \
0  xS5QSLNJ3SOVCFUK            Pat Benatar  In The Heat Of The Night   
1  lYWQPI2TFoh8nSTC                   Blur                    Tender   
2  hMoLbFgENhtFTrS7             Lucy Pearl                  Everyday   
3  FiXIOm6C8woWhma4         Bat for Lashes                    Lilies   
4  32m5suoC94ytD8Ed          Ariana Grande                   7 rings   
5  RcOEUo7Nkj3CIPbF        Scissor Sisters           Only the Horses   
6  5PsAhyXTxQowhIUu  Donavon Frankenreiter                On My Mind   
7  CVirHSstAw0VXT6n          Big Time Rush                   Oh Yeah   
8  aUeTyLDIpE8XdDK9        Tracy Ate A Bug            Schwarze Witwe   
9  OnL2vyNc6akjB3kD               Paramore             Feeling Sorry   

                         album_name  
0          In The Heat Of The Night  
1                                13  
2                        Lucy Pearl  
3  The Haunted Man (Deluxe Version)  
4                

---
#### Subtask 2: Compute Accuracy Metrics for Baseline

Using genre as a relevance criterion, we need to extract the top genre. (a retrieved song is considered relevant if its top genre matches the top genre of the query)

In [112]:
def get_top_genre(genres):
    return genres[0] if genres else None

catalog_df['top_genre'] = catalog_df['genre'].apply(get_top_genre)

#example
print(catalog_df[['id', 'top_genre']].head())

                 id     top_genre
0  01rMxQv6vhyE1oQX          rock
1  02ZnlCGZEbkfCDxo           pop
2  04OjszRi9rC5BlHC  experimental
3  04iitW3ffa0mhpx3           pop
4  04xUDjAYC14jsHyH          punk


To evaluate the performance of a retrieval system, we use several key metrics, here are they:

---
##### 1. Precision@k

Precision@k measures the proportion of relevant items in the top $k$ retrieved items. It is defined as:

$$
\text{Precision@k} = \frac{\text{Number of Relevant Retrieved Items in Top } k}{k}
$$

Precision@k focuses on the accuracy of the retrieved results in the top $k$ and is not influenced by the total number of relevant items in the dataset.

In [ ]:
def precision_at_k(retrieved_ids, relevant_ids, k=10):
    """
    Computes Precision@k
    """
    retrieved_at_k = retrieved_ids[:k]
    relevant_retrieved = set(retrieved_at_k) & set(relevant_ids)
    return len(relevant_retrieved) / k

---
##### 2. Recall@k

Recall@k measures the proportion of all relevant items that are retrieved in the top $k$ results. It is defined as:

$$
\text{Recall@k} = \frac{\text{Number of Relevant Retrieved Items in Top } k}{\text{Total Number of Relevant Items}}
$$

This metric emphasizes the completeness of retrieval by accounting for all relevant items.

In [ ]:
def recall_at_k(retrieved_ids, relevant_ids, k=10):
    """
    Computes Recall@k
    """
    retrieved_at_k = retrieved_ids[:k]
    relevant_retrieved = set(retrieved_at_k) & set(relevant_ids)
    total_relevant = len(relevant_ids)
    return len(relevant_retrieved) / total_relevant if total_relevant > 0 else 0.0

---
##### 3. NDCG@k (Normalized Discounted Cumulative Gain)

NDCG@k evaluates the ranking quality of the retrieved items by considering both relevance and position. It is calculated as:

$$
\text{NDCG@k} = \frac{\text{DCG@k}}{\text{IDCG@k}}
$$

Where the Discounted Cumulative Gain (DCG) at $k$ is:

$$
\text{DCG@k} = \sum_{i=1}^{k} \frac{2^{\text{rel}_i} - 1}{\log_2(i + 1)}
$$

The Ideal DCG (IDCG) is the DCG of an ideal ranking, where the most relevant items are ranked highest:

$$
\text{IDCG@k} = \sum_{i=1}^{k} \frac{2^{\text{rel}_i^\text{ideal}} - 1}{\log_2(i + 1)}
$$

NDCG@k normalizes the ranking score, making it comparable across different queries.

In [ ]:
def ndcg_at_k(retrieved_ids, relevant_ids, k=10):
    """
    Computes NDCG@k
    """
    retrieved_at_k = retrieved_ids[:k]
    relevance = [1 if track_id in relevant_ids else 0 for track_id in retrieved_at_k]
    # NDCG expects a list of relevance lists
    return ndcg_score([relevance], [relevance], k=k)

---
##### 4. MRR (Mean Reciprocal Rank)

The Mean Reciprocal Rank (MRR) measures the average rank position of the first relevant item across multiple queries. It is defined as:

$$
\text{MRR} = \frac{1}{Q} \sum_{q=1}^{Q} \frac{1}{\text{rank}_q}
$$

Where:
- $Q$ is the number of queries.
- $\text{rank}_q$ is the position of the first relevant item for the $q^\text{th}$ query.

MRR focuses on the speed at which the first relevant item is retrieved.

In [27]:
def mrr(retrieved_ids, relevant_ids):
    """
    Computes Reciprocal Rank
    """
    for rank, track_id in enumerate(retrieved_ids, start=1):
        if track_id in relevant_ids:
            return 1.0 / rank
    return 0.0

---
##### Evaluate the Random Baseline

In [60]:
def evaluate_random_baseline(catalog_df, N=10):
    """
    Evaluates the random baseline retrieval system across all queries.
    
    Parameters:
    - catalog_df (pd.DataFrame): The catalog containing all tracks with 'id' and 'top_genre'.
    - N (int): Number of tracks to retrieve per query.
    
    Returns:
    - metrics_dict (dict): Dictionary containing average Precision@10, Recall@10, NDCG@10, and MRR.
    """
    precisions = []
    recalls = []
    ndcgs = []
    mrrs = []
    
    #iterate over each track as a query
    for index, query_track in catalog_df.iterrows():
        query_id = query_track['id']
        query_genre = query_track['top_genre']
        
        #if top_genre doesn't exist
        if not query_genre:
            continue
        
        #retrieve N tracks
        retrieved_tracks = random_retrieval(query_id, catalog_df, N)
        retrieved_ids = retrieved_tracks['id'].tolist()
        
        #define relevant
        relevant_ids = catalog_df[catalog_df['top_genre'] == query_genre]['id'].tolist()
        
        #metrics
        p = precision_at_k(retrieved_ids, relevant_ids, k=N)
        r = recall_at_k(retrieved_ids, relevant_ids, k=N)
        ndcg = ndcg_at_k(retrieved_ids, relevant_ids, k=N)
        rr = mrr(retrieved_ids, relevant_ids)
        
        precisions.append(p)
        recalls.append(r)
        ndcgs.append(ndcg)
        mrrs.append(rr)
        
    #aggregate metrics
    metrics_dict = {
        'Precision@10': np.mean(precisions),
        'Recall@10': np.mean(recalls),
        'NDCG@10': np.mean(ndcgs),
        'MRR': np.mean(mrrs)
    }
    
    return metrics_dict

metrics = evaluate_random_baseline(catalog_df, N=10)

#results
print("Random Baseline Metrics:")
print(f"Precision@10: {metrics['Precision@10']:.4f}")
print(f"Recall@10: {metrics['Recall@10']:.4f}")
print(f"NDCG@10: {metrics['NDCG@10']:.4f}")
print(f"MRR: {metrics['MRR']:.4f}")

Random Baseline Metrics:
Precision@10: 0.0490
Recall@10: 0.0019
NDCG@10: 0.3071
MRR: 0.1114


---

The results align with expectations for a random baseline:
- Low precision and recall due to the absence of relevance-based ranking.
- Not that low NDCG and MRR values arise from occasional favorable random placements of relevant items.

These metrics serve as a benchmark to compare against more sophisticated retrieval systems.

---
#### Subtask 3: Implement and Evaluate a Text-Based Retrieval System Using TF-IDF

In [ ]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
from scipy.sparse import csr_matrix
import numpy as np

# Define file paths
DATA_DIR = 'data/'  # Update this path if your data is in a different directory
INFORMATION_FILE = DATA_DIR + 'id_information_mmsr.tsv'
GENRES_FILE = DATA_DIR + 'id_genres_mmsr.tsv'
LYRICS_TFIDF_FILE = DATA_DIR + 'id_lyrics_tf-idf_mmsr.tsv'

genres_df['genre'] = genres_df['genre'].apply(parse_genres)

# Merge information and genres to create catalog
catalog_df = pd.merge(information_df, genres_df, on='id', how='left')

# Handle missing genres by assigning an empty list
catalog_df['genre'] = catalog_df['genre'].apply(lambda x: x if isinstance(x, list) else [])

catalog_df['top_genre'] = catalog_df['genre'].apply(get_top_genre)

# Load precomputed TF-IDF vectors
tfidf_df = pd.read_csv(LYRICS_TFIDF_FILE, sep='\t')
print("Loaded tfidf_df successfully.")


# Inspect the first few rows of tfidf_df
print("\nSample of tfidf_df:")
print(tfidf_df.head())

# Check if 'song' exists in tfidf_df to avoid duplication during merge
if 'song' in tfidf_df.columns:
    # Rename 'song' in tfidf_df to prevent duplication
    tfidf_df = tfidf_df.rename(columns={'song': 'tfidf_song'})
    print("\nRenamed 'song' column in tfidf_df to 'tfidf_song' to avoid duplication.")

# Merge TF-IDF vectors with catalog on 'id'
catalog_df = pd.merge(catalog_df, tfidf_df, on='id', how='left')

# Inspect columns after merging
print("\nColumns in catalog_df after merging TF-IDF:")
print(catalog_df.columns.tolist())

# Identify TF-IDF feature columns (exclude 'id' and metadata columns)
metadata_columns = ['artist', 'song', 'album_name', 'genre', 'top_genre']
tfidf_feature_columns = [col for col in tfidf_df.columns if col != 'id' and col != 'tfidf_song']

# Drop rows with any NaN in TF-IDF features to ensure complete data
initial_catalog_size = len(catalog_df)
catalog_df = catalog_df.dropna(subset=tfidf_feature_columns).reset_index(drop=True)
final_catalog_size = len(catalog_df)
dropped_rows = initial_catalog_size - final_catalog_size
print(f"\nDropped {dropped_rows} tracks due to missing TF-IDF vectors.")
print(f"Catalog size after merging TF-IDF vectors: {final_catalog_size}")

# Display the first few rows with relevant columns
print("\nSample of catalog_df:")
print(catalog_df[['id', 'artist', 'song', 'top_genre']].head())


Loaded tfidf_df successfully.

Sample of tfidf_df:
                 id  abl  accept    across  act  addict    afraid  age  \
0  h48f46ZsT9h0Z5Dm  0.0     0.0  0.000000  0.0     0.0  0.000000  0.0   
1  PV5EXN6AIVBqvsLO  0.0     0.0  0.000000  0.0     0.0  0.327025  0.0   
2  eFEY5JiDF3ZLpXBZ  0.0     0.0  0.143314  0.0     0.0  0.000000  0.0   
3  VAWiymoCIYxhae3J  0.0     0.0  0.000000  0.0     0.0  0.000000  0.0   
4  2H91WLAd7ZZJvAiw  0.0     0.0  0.000000  0.0     0.0  0.000000  0.0   

        ago   ah  ...  yea      yeah  year  yellow  yes  yesterday  yet   yo  \
0  0.000000  0.0  ...  0.0  0.000000   0.0     0.0  0.0   0.149783  0.0  0.0   
1  0.000000  0.0  ...  0.0  0.000000   0.0     0.0  0.0   0.000000  0.0  0.0   
2  0.000000  0.0  ...  0.0  0.042526   0.0     0.0  0.0   0.000000  0.0  0.0   
3  0.109514  0.0  ...  0.0  0.000000   0.0     0.0  0.0   0.000000  0.0  0.0   
4  0.000000  0.0  ...  0.0  0.084732   0.0     0.0  0.0   0.000000  0.0  0.0   

   young  youth  
0    

In [93]:
from scipy.sparse import csr_matrix

# Separate track IDs and TF-IDF features
track_ids = catalog_df['id'].tolist()

# Extract TF-IDF features (ensure 'tfidf_song' is excluded)
tfidf_features = catalog_df.drop(columns=['id', 'artist', 'song', 'album_name', 'genre', 'top_genre'])

# Convert to a sparse matrix for efficient storage and computation
tfidf_matrix = csr_matrix(tfidf_features.values)

print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}")


TF-IDF matrix shape: (5148, 1000)


In [81]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def tfidf_retrieval(query_track_id, id_to_index, tfidf_matrix, track_ids, catalog_df, N=10):
    """
    Retrieves N tracks most similar to the query track based on TF-IDF cosine similarity.

    Parameters:
    - query_track_id (str): The ID of the query track.
    - id_to_index (dict): Mapping from track ID to matrix index.
    - tfidf_matrix (csr_matrix): Sparse matrix of TF-IDF vectors.
    - track_ids (list): List of track IDs corresponding to tfidf_matrix rows.
    - catalog_df (pd.DataFrame): DataFrame containing track metadata.
    - N (int): Number of tracks to retrieve.

    Returns:
    - retrieved_tracks_df (pd.DataFrame): DataFrame of retrieved tracks sorted by similarity.
    """
    # Check if query_track_id exists
    if query_track_id not in id_to_index:
        print(f"Query track ID {query_track_id} not found.")
        return pd.DataFrame()
    
    # Get the index of the query track
    query_index = id_to_index[query_track_id]
    
    # Get the TF-IDF vector for the query track
    query_vector = tfidf_matrix[query_index]
    
    # Compute cosine similarity between the query and all tracks
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Exclude the query track itself by setting its similarity to -1
    similarities[query_index] = -1
    
    # Get the indices of the top N similar tracks
    top_indices = similarities.argsort()[-N:][::-1]
    
    # Retrieve the corresponding track IDs and similarity scores
    retrieved_ids = [track_ids[i] for i in top_indices]
    retrieved_scores = [similarities[i] for i in top_indices]
    
    # Create a DataFrame for retrieved tracks
    retrieved_tracks_df = pd.DataFrame({
        'id': retrieved_ids,
        'similarity': retrieved_scores
    })
    
    # Merge with catalog_df to get track details
    retrieved_tracks_df = pd.merge(retrieved_tracks_df, catalog_df[['id', 'artist', 'song', 'album_name']], on='id', how='left')
    
    return retrieved_tracks_df


In [84]:
# Evaluation function
def evaluate_tfidf_retrieval(catalog_df, tfidf_matrix, track_ids, id_to_index, N=10):
    """
    Evaluates the TF-IDF retrieval system across all queries.

    Parameters:
    - catalog_df (pd.DataFrame): DataFrame containing all tracks with 'id' and 'top_genre'.
    - tfidf_matrix (csr_matrix): Sparse matrix of TF-IDF vectors.
    - track_ids (list): List of track IDs.
    - id_to_index (dict): Mapping from track ID to index.
    - N (int): Number of tracks to retrieve per query.

    Returns:
    - metrics_dict (dict): Dictionary containing average Precision@10, Recall@10, NDCG@10, and MRR.
    """
    precisions = []
    recalls = []
    ndcgs = []
    mrrs = []
    
    total_queries = len(catalog_df)
    processed_queries = 0
    
    for index, query_track in catalog_df.iterrows():
        query_id = query_track['id']
        query_genre = query_track['top_genre']
        
        # Skip if no top genre
        if not query_genre:
            continue
        
        # Retrieve N tracks using TF-IDF retrieval
        retrieved_tracks = tfidf_retrieval(
            query_track_id=query_id,
            id_to_index=id_to_index,
            tfidf_matrix=tfidf_matrix,
            track_ids=track_ids,
            catalog_df=catalog_df,
            N=N
        )
        
        # If retrieval failed or no tracks retrieved, skip
        if retrieved_tracks.empty:
            continue
        
        retrieved_ids = retrieved_tracks['id'].tolist()
        
        # Define relevant tracks based on top genre
        relevant_ids = catalog_df[catalog_df['top_genre'] == query_genre]['id'].tolist()
        
        # Compute metrics
        p = precision_at_k(retrieved_ids, relevant_ids, k=N)
        r = recall_at_k(retrieved_ids, relevant_ids, k=N)
        ndcg = ndcg_at_k(retrieved_ids, relevant_ids, k=N)
        rr = mrr_metric(retrieved_ids, relevant_ids)
        
        precisions.append(p)
        recalls.append(r)
        ndcgs.append(ndcg)
        mrrs.append(rr)
        
        processed_queries += 1
        if processed_queries % 500 == 0:
            print(f"Processed {processed_queries}/{total_queries} queries")
    
    # Aggregate metrics
    metrics_dict = {
        'Precision@10': np.mean(precisions) if precisions else 0,
        'Recall@10': np.mean(recalls) if recalls else 0,
        'NDCG@10': np.mean(ndcgs) if ndcgs else 0,
        'MRR': np.mean(mrrs) if mrrs else 0
    }
    
    return metrics_dict

# Main execution
if __name__ == "__main__":
    print("\nStarting evaluation of TF-IDF Retrieval System...\n")
    metrics = evaluate_tfidf_retrieval(catalog_df, tfidf_matrix, track_ids, id_to_index, N=10)
    
    print("\nTF-IDF Retrieval Metrics:")
    print(f"Precision@10: {metrics['Precision@10']:.4f}")
    print(f"Recall@10: {metrics['Recall@10']:.4f}")
    print(f"NDCG@10: {metrics['NDCG@10']:.4f}")
    print(f"MRR: {metrics['MRR']:.4f}")



Starting evaluation of TF-IDF Retrieval System...

Processed 500/5148 queries
Processed 1000/5148 queries
Processed 1500/5148 queries
Processed 2000/5148 queries
Processed 2500/5148 queries
Processed 3000/5148 queries
Processed 3500/5148 queries
Processed 4000/5148 queries
Processed 4500/5148 queries
Processed 5000/5148 queries

TF-IDF Retrieval Metrics:
Precision@10: 0.0739
Recall@10: 0.0037
NDCG@10: 0.2040
MRR: 0.1563


---
##### Visualizations

# 2. Beyond-Accuracy and Trade-Offs:
#### Additional Modalities:
- Select one text feature (e.g., word2vec or BERT).
- Add two audio and two visual features with respective similarity metrics.
- Evaluate accuracy metrics (same as Task 1).
#### Beyond-Accuracy Metrics:
- Compute retrievability/coverage, diversity, and popularity metrics.
#### Optimization and Trade-Offs:
- Implement diversity optimization.
- Study trade-offs between NDCG and diversity.

# 3. Multimodal and Creative:
#### New Retrieval System:
- Devise a novel approach (e.g., LTR or LLMs).
#### Fusion Methods:
- Combine systems using early and late fusion techniques.
#### Final System:
- Evaluate accuracy and beyond-accuracy metrics.
- Build a comprehensive, user-friendly UI.
